#### The following analysis was performed in collaboration with Hayley B and Dr. Yin. The samples were collected on 7/23/20 and anayzed 7/31/20. 

In [23]:
import pandas as pd
import altair as alt
from importlib import reload
import ntpath
import os
import ms1
reload(ms1)
from ms1 import PeakListFile

In [58]:
direct = r"D:\Research\MS Data\Orbitrap Elite\20200731_ZipChip\Excel"
all_files = []
for root, dirs, files in os.walk(direct):
    for file in files:
        filename = os.path.join(root, file)
        all_files.append(filename)

In [59]:
import pandas as pd
from importlib import reload
import ntpath
import ms1
reload(ms1)
from ms1 import PeakListFile
test = all_files[0]


d = PeakListFile(test)
print(d)
d.combine_sheets(["Histidine", "Phenylalanine"], "Histidine", 1e-3)
d.correct_dilution("Histidine", 400)

PeakListFile object instanitated from D:\Research\MS Data\Orbitrap Elite\20200731_ZipChip\Excel\F_None_0a.xlsx
combine complete


In [60]:
d.data

,Sample,Fraction,Run,Identity,Area,Apex RT,Area Ratio,Concentration,Corrected Concentration
38,F_None_0,a,run1,Histidine,9.551345e+06,1.02,1.000000,0.001000,0.001000
68,F_None_0,a,run1,Phenylalanine,4.412327e+07,1.68,4.619587,0.004620,1.847835
34,F_None_0,a,run2,Histidine,9.444642e+06,1.00,1.000000,0.001000,0.001000
59,F_None_0,a,run2,Phenylalanine,4.366160e+07,1.66,4.622896,0.004623,1.849159
41,F_None_0,a,run3,Histidine,9.714563e+06,1.01,1.000000,0.001000,0.001000
71,F_None_0,a,run3,Phenylalanine,4.369794e+07,1.65,4.498189,0.004498,1.799276


In [66]:
fdf = pd.DataFrame()
gdf = pd.DataFrame()
for file in all_files:
    try:
        p = PeakListFile(file)
        if ntpath.basename(file).startswith('F'):
            p.combine_sheets(['Histidine', "Phenylalanine"], standard='Histidine',
                             std_conc=10e-6)
            p.correct_dilution(standard='Histidine', dil_factor=400)
            if fdf.empty:
                fdf = p.data
            else:
                fdf = pd.concat([fdf, p.data])
        elif ntpath.basename(file).startswith('G'):
            p.combine_sheets(['Histidine', 'Glycine'], standard='Histidine',
                             std_conc=10e-6)
            p.correct_dilution(standard='Histidine', dil_factor=500)
            if gdf.empty:
                gdf = p.data
            else:
                gdf = pd.concat([gdf, p.data])
    except:
        print(file)

combine complete
combine complete
combine complete
combine complete
combine complete
combine complete
combine complete
combine complete
combine complete
combine complete
combine complete
combine complete
combine complete
combine complete
combine complete
combine complete
combine complete
combine complete
combine complete
combine complete
combine complete
combine complete
combine complete
combine complete
combine complete
combine complete
combine complete
combine complete
combine complete
combine complete
combine complete
combine complete


In [67]:
# make plot for f samples

source = fdf[fdf['Identity']=="Phenylalanine"]

bars = alt.Chart(source).mark_bar(size=15).encode(
    x="Fraction:O",
    y=alt.Y('mean(Corrected Concentration):Q', title="Mean Concentration"),
    color="Identity:O"
).properties()

error_bars = alt.Chart(source).mark_errorbar(extent="stdev").encode(
    x="Fraction:O",
    y="Corrected Concentration:Q"
)

alt.layer(bars, error_bars).facet(
    column="Sample:O"
)

alt.FacetChart(...)

In [68]:
# make plot for G samples
source = gdf[gdf['Identity']=="Glycine"]

bars = alt.Chart(source).mark_bar(size=15).encode(
    x="Fraction:O",
    y=alt.Y('mean(Corrected Concentration):Q', title="Mean Concentration"),
    color="Identity:O"
).properties()

error_bars = alt.Chart(source).mark_errorbar(extent="stdev").encode(
    x="Fraction:O",
    y="Corrected Concentration:Q"
)

alt.layer(bars, error_bars).facet(
    column="Sample:O"
)

alt.FacetChart(...)

In [69]:
# compare His in all samples to look for aberrations

data = pd.concat([fdf[fdf.Identity=="Histidine"], gdf[gdf.Identity=="Histidine"]])

bars = alt.Chart().mark_bar(size=12).encode(
    x="Fraction:O",
    y=alt.Y("mean(Area):Q", title="Mean Area")
)

error_bars = alt.Chart().mark_errorbar(extent="stdev").encode(
    x="Fraction:O",
    y="Area:Q",
)

alt.layer(bars, error_bars, data=data).facet(
    column="Sample:O"
)

alt.FacetChart(...)

In [47]:
# create average dataframes
# averaged based on having same sample, fraction, and identity

fdf_avg = fdf.groupby(['Sample', 'Fraction', 'Identity']).mean()
gdf_avg = gdf.groupby(['Sample', 'Fraction', 'Identity']).mean()

# create std_dev dataframes
fdf_std = fdf.groupby(['Sample', 'Fraction', 'Identity']).std()
fdf_std.columns = [column+'_StdDev' for column in fdf_std.columns]
gdf_std = gdf.groupby(['Sample', 'Fraction', 'Identity']).std()
gdf_std.columns = [column+'_StdDev' for column in gdf_std.columns]

In [48]:
gdf_std

Area_StdDev  Apex RT_StdDev  Area Ratio_StdDev  \
Sample   Fraction Identity                                                     
G_None_0 a        Glycine    3.235934e+05        0.010000           0.029573   
                  Histidine  1.697277e+05        0.011547           0.000000   
         b        Glycine    2.638325e+05        0.005774           0.017919   
                  Histidine  1.919968e+05        0.005774           0.000000   
G_None_1 a        Glycine    5.092082e+05        0.000000           0.009615   
                  Histidine  2.989901e+05        0.005774           0.000000   
         b        Glycine    4.128428e+05        0.005774           0.053085   
                  Histidine  1.804332e+05        0.000000           0.000000   
G_None_2 a        Glycine    1.253643e+06        0.015275           0.036939   
                  Histidine  1.153045e+06        0.015275           0.000000   
         b        Glycine    3.639862e+05        0.000000           0.013829   
                  Histidine  3.266437e+05        0.000000           0.000000   
G_None_3 a        Glycine    3.904096e+05        0.005774           0.013536   
                  Histidine  2.740247e+05        0.005774           0.000000   
         b        Glycine    5.152868e+05        0.017321           0.037749   
                  Histidine  4.649334e+05        0.011547           0.000000   
G_TP_0   a        Glycine    6.386493e+05        0.005774           0.023173   
                  Histidine  3.455887e+05        0.000000           0.000000   
         b        Glycine    7.450335e+05        0.010000           0.082367   
                  Histidine  4.222598e+05        0.010000           0.000000   
G_TP_1   a        Glycine    7.555896e+05        0.011402           0.061196   
                  Histidine  5.298805e+05        0.010954           0.000000   
         b        Glycine    3.191846e+05        0.005774           0.000754   
                  Histidine  1.923012e+05        0.005774           0.000000   
G_TP_2   a        Glycine    4.206407e+05        0.011547           0.047248   
                  Histidine  2.363227e+05        0.010000           0.000000   
         b        Glycine    8.864365e+05        0.015275           0.051883   
                  Histidine  3.324616e+05        0.015275           0.000000   
G_TP_3   a        Glycine    2.588045e+05        0.000000           0.047271   
                  Histidine  2.468207e+05        0.000000           0.000000   
         b        Glycine    2.925573e+05        0.000000           0.023827   
                  Histidine  2.318975e+05        0.005774           0.000000   

                             Concentration_StdDev  \
Sample   Fraction Identity                          
G_None_0 a        Glycine            2.957307e-05   
                  Histidine          0.000000e+00   
         b        Glycine            1.791922e-05   
                  Histidine          0.000000e+00   
G_None_1 a        Glycine            9.615409e-06   
                  Histidine          0.000000e+00   
         b        Glycine            5.308502e-05   
                  Histidine          0.000000e+00   
G_None_2 a        Glycine            3.693894e-05   
                  Histidine          0.000000e+00   
         b        Glycine            1.382878e-05   
                  Histidine          0.000000e+00   
G_None_3 a        Glycine            1.353596e-05   
                  Histidine          0.000000e+00   
         b        Glycine            3.774898e-05   
                  Histidine          0.000000e+00   
G_TP_0   a        Glycine            2.317294e-05   
                  Histidine          0.000000e+00   
         b        Glycine            8.236748e-05   
                  Histidine          0.000000e+00   
G_TP_1   a        Glycine            6.119597e-05   
                  Histidine          0.000000e+00   
         b        Glycine            

In [49]:
with pd.ExcelWriter('DataAnalysis_GFSamples_20200731.xlsx') as writer:
    fdf.to_excel(writer, sheet_name="Phe")
    gdf.to_excel(writer, sheet_name="Gly")
    fdf_avg.to_excel(writer, sheet_name="Phe_Avg")
    fdf_std.to_excel(writer, sheet_name="Phe_StdDev")
    gdf_avg.to_excel(writer, sheet_name="Glycine_Avg")
    gdf_std.to_excel(writer, sheet_name="Gly_StdDev")

In [86]:
## Analysis for 08/01 Samples
import ms1
reload(ms1)
from ms1 import PeakListFile

direc = r"D:\Research\MS Data\Orbitrap Elite\20200801_ZipChip\Excel\Correct"
all_files = []
for root, dirs, files in os.walk(direc):
    for file in files:
        filename = os.path.join(root, file)
        all_files.append(filename)

samp_dil = {'1':10_000, '2':2_000, '3':100, '4':10}
cutoffs = {'F1': 4, 'F2':5, 'G4':10}

fdf = pd.DataFrame()
gdf = pd.DataFrame()
for file in all_files:
    try:
        p = PeakListFile(file)
        basename = ntpath.basename(file).split('.')[0]
        root, suff = basename[0], basename[1]
        if root == 'F':
            p.combine_sheets(['Histidine', "Phenylalanine"], standard='Histidine',
                             std_conc=10e-6, cutoff=cutoffs[basename])
            p.correct_dilution(standard='Histidine', dil_factor=samp_dil[suff])
            if fdf.empty:
                fdf = p.data
            else:
                fdf = pd.concat([fdf, p.data])
        elif root == 'G':
            p.combine_sheets(['Histidine', 'Glycine'], standard='Histidine',
                             std_conc=10e-6, cutoff=cutoffs[basename])
            p.correct_dilution(standard='Histidine', dil_factor=samp_dil[suff])
            if gdf.empty:
                gdf = p.data
            else:
                gdf = pd.concat([gdf, p.data])
    except:
        print(file)

combine complete
combine complete
combine complete


In [87]:
source1 = fdf[(fdf['Identity']=="Phenylalanine") & (fdf['Fraction']=="1")]
source2 = fdf[(fdf['Identity']=="Phenylalanine") & (fdf['Fraction']=="2")]

bars = alt.Chart().mark_bar(size=15).encode(
    x="Sample:O",
    y=alt.Y('mean(Corrected Concentration):Q', title="Mean Concentration"),
    color="Identity:O"
).properties()

error_bars = alt.Chart().mark_errorbar(extent="stdev").encode(
    x="Sample:O",
    y="Corrected Concentration:Q"
)

# alt.layer(bars, error_bars, data=source1).facet(
#     column="Fraction:O"
# ).save("F1.png", scale_factor=10)
# alt.layer(bars, error_bars, data=source2).facet(
#     column="Fraction:O"
# ).save("F2.png", scale_factor=10)

In [88]:
source = gdf[gdf['Identity']=="Glycine"]

bars = alt.Chart(source).mark_bar(size=15).encode(
    x="Fraction:O",
    y=alt.Y('mean(Corrected Concentration):Q', title="Mean Concentration"),
    color="Identity:O"
).properties()

error_bars = alt.Chart(source).mark_errorbar(extent="stdev").encode(
    x="Fraction:O",
    y="Corrected Concentration:Q"
)

# alt.layer(bars, error_bars).facet(
#     column="Sample:O"
# ).save('G4.png', scale_factor=10)

In [89]:
fdf_avg = fdf.groupby(['Sample', 'Fraction', 'Identity']).mean()
gdf_avg = gdf.groupby(['Sample', 'Fraction', 'Identity']).mean()

# create std_dev dataframes
fdf_std = fdf.groupby(['Sample', 'Fraction', 'Identity']).std()
fdf_std.columns = [column+'_StdDev' for column in fdf_std.columns]
gdf_std = gdf.groupby(['Sample', 'Fraction', 'Identity']).std()
gdf_std.columns = [column+'_StdDev' for column in gdf_std.columns]

In [92]:
with pd.ExcelWriter('DataAnalysis_GFStandards_20200801.xlsx') as writer:
    fdf.to_excel(writer, sheet_name="Phe")
    gdf.to_excel(writer, sheet_name="Gly")
    fdf_avg.to_excel(writer, sheet_name="Phe_Avg")
    fdf_std.to_excel(writer, sheet_name="Phe_StdDev")
    gdf_avg.to_excel(writer, sheet_name="Glycine_Avg")
    gdf_std.to_excel(writer, sheet_name="Gly_StdDev")